# Binary Classification Example using GBTClassifier

The Spark MLlib Pipelines API provides higher-level API built on top of DataFrames for constructing ML pipelines.
You can read more about the Pipelines API in the [MLlib programming guide](https://spark.apache.org/docs/latest/ml-guide.html).


## Dataset Review

The Adult dataset is publicly available at the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Adult).
This data derives from census data and consists of information about 48842 individuals and their annual income.
You can use this information to predict if an individual earns **<=50K or >50k** a year.
The dataset consists of both numeric and categorical variables.

Attribute Information:

- age: continuous
- workclass: Private,Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked
- fnlwgt: continuous
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc...
- education-num: continuous
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent...
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners...
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black
- sex: Female, Male
- capital-gain: continuous
- capital-loss: continuous
- hours-per-week: continuous
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany...

Target/Label: - <=50K, >50K


## Load Data

The Adult dataset is available in databricks-datasets. Read in the data using the CSV data source for Spark and rename the columns appropriately.

In [0]:
%fs ls databricks-datasets/adult/adult.data

In [0]:
%fs head databricks-datasets/adult/adult.data

In [0]:
from pyspark.sql.types import DoubleType, StringType, StructField, StructType

schema = StructType([
  StructField("age", DoubleType(), False),
  StructField("workclass", StringType(), False),
  StructField("fnlwgt", DoubleType(), False),
  StructField("education", StringType(), False),
  StructField("education_num", DoubleType(), False),
  StructField("marital_status", StringType(), False),
  StructField("occupation", StringType(), False),
  StructField("relationship", StringType(), False),
  StructField("race", StringType(), False),
  StructField("sex", StringType(), False),
  StructField("capital_gain", DoubleType(), False),
  StructField("capital_loss", DoubleType(), False),
  StructField("hours_per_week", DoubleType(), False),
  StructField("native_country", StringType(), False),
  StructField("income", StringType(), False)
])

dataset = spark.read.format("csv").schema(schema).load("/databricks-datasets/adult/adult.data")
cols = dataset.columns

In [0]:
display(dataset)

## Preprocess Data

To use algorithms like Logistic Regression, you must first convert the categorical variables in the dataset into numeric variables.
There are two ways to do this.

* Category Indexing

  This is basically assigning a numeric value to each category from {0, 1, 2, ...numCategories-1}.
  This introduces an implicit ordering among your categories, and is more suitable for ordinal variables (eg: Poor: 0, Average: 1, Good: 2)

* One-Hot Encoding

  This converts categories into binary vectors with at most one nonzero value (eg: (Blue: [1, 0]), (Green: [0, 1]), (Red: [0, 0]))

This notebook uses a combination of [StringIndexer] and, depending on your Spark version, either [OneHotEncoder] or [OneHotEncoderEstimator] to convert the categorical variables.
`OneHotEncoder` and `OneHotEncoderEstimator` return a [SparseVector]. 

Since there is more than one stage of feature transformations, use a [Pipeline] to tie the stages together.
This simplifies the code.

[StringIndexer]: http://spark.apache.org/docs/latest/ml-features.html#stringindexer
[OneHotEncoderEstimator]: https://spark.apache.org/docs/2.4.5/api/python/pyspark.ml.html?highlight=one%20hot%20encoder#pyspark.ml.feature.OneHotEncoderEstimator
[SparseVector]: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.linalg.SparseVector.html#pyspark.ml.linalg.SparseVector
[Pipeline]: https://spark.apache.org/docs/latest/ml-pipeline.html#ml-pipelines
[OneHotEncoder]: https://spark.apache.org/docs/latest/ml-features.html#onehotencoder

In [0]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler

from distutils.version import LooseVersion

categoricalColumns = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "sex", "native_country"]
stages = [] # stages in Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    if LooseVersion(pyspark.__version__) < LooseVersion("3.0"):
        from pyspark.ml.feature import OneHotEncoderEstimator
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    else:
        from pyspark.ml.feature import OneHotEncoder
        encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Add stages.  These are not run here, but will run all at once later on.
    stages.append(stringIndexer)
    stages.append(encoder)

The above code basically indexes each categorical column using the `StringIndexer`,
and then converts the indexed categories into one-hot encoded variables.
The resulting output has the binary vectors appended to the end of each row.

Use the `StringIndexer` again to encode labels to label indices.

In [0]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol="income", outputCol="label")
stages.append(label_stringIdx)

Use a `VectorAssembler` to combine all the feature columns into a single vector column.
This includes both the numeric columns and the one-hot encoded binary vector columns in the dataset.

In [0]:
# Transform all features into a vector using VectorAssembler
numericCols = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler()
assembler.setInputCols(assemblerInputs)
assembler.setOutputCol("features")
stages.append(assembler)

Run the stages as a Pipeline. This puts the data through all of the feature transformations in a single call.

In [0]:
partialPipeline = Pipeline().setStages(stages)
last_stage_output_col = partialPipeline.getStages()[-1].getOutputCol()
last_stage_output_col
pipelineModel = partialPipeline.fit(dataset)
pipelineModel.stages[-1].setOutputCol("features")
preppedDataDF = pipelineModel.transform(dataset)

In [0]:
# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = preppedDataDF.select(selectedcols)
display(dataset)

In [0]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(testData.count())

## Fit and Evaluate Models

We are using the Gradient Boosting Classifier from MLlib's classification algorithms:
  - GBTClassifier (Gradient Boosted Tree Classifier)

## Gradient Boosting Classifier

You can read more about [Gradient Boosting Classifier] from the [classification and regression] section of MLlib Programming Guide.
In the Pipelines API, Gradient Boosting Classifier is a powerful ensemble learning method that builds a series of decision trees sequentially, with each tree correcting errors made by the previous ones.

[classification and regression]: https://spark.apache.org/docs/latest/ml-classification-regression.html
[Gradient Boosting Classifier]: https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-classifier

In [0]:
from pyspark.ml.classification import GBTClassifier

# Create initial GBTClassifier model
# The income label is a categorical variable with two values: <=50K and >50K
# This has to be passed to the GBTClassifier
gbtc = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
gbtc = gbtc.fit(trainingData)

In [0]:
# Make predictions on test data using the transform() method.
# GBTClassifier.transform() will only use the 'features' column.
predictions = gbtc.transform(testData)

In [0]:
# View model's predictions and probabilities of each prediction class
# You can select any columns in the above schema to view as well
selected = predictions.select("label", "prediction", "probability")
display(selected)

### Use `BinaryClassificationEvaluator` and `MulticlassClassificationEvaluator` to evaluate the model.

`BinaryClassificationEvaluator` to evaluate the model's [areaUnderROC] metric.

[areaUnderROC]: https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve

`MulticlassClassificationEvaluator` to evaluate the model's [F1](https://en.wikipedia.org/wiki/F-score) metric 

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Evaluate model - AUC
auc_evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
auc = auc_evaluator.evaluate(predictions)
print(f"Area under ROC curve: {auc}")

# Evaluate model - F1 Score
f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = f1_evaluator.evaluate(predictions)
print(f"F1 Score: {f1}")